# Converting the MNE "somato-data" to BIDS format

## Requirements

The python dependencies to run this notebook are:
- jupyter
- numpy
- scipy
- mne<0.19
- mne-bids
- nibabel


## Helpful links

- https://github.com/mne-tools/mne-bids

## Copyright
```Text
# Authors: Stefan Appelhoff <stefan.appelhoff@mailbox.org>
#
# License: BSD (3-clause)
```


In [1]:
import os
import os.path as op
import shutil as sh
import json
import warnings

import numpy as np
import mne
import mne.datasets.somato as somato
from mne_bids import write_raw_bids, make_bids_basename, write_anat
from mne_bids.utils import print_dir_tree

# Convert to BIDS

In [2]:
# Path to the raw data of somato dataset
fif_path = op.join(somato.data_path(), 'MEG', 'somato')

In [3]:
# Load the raw data file
raw = mne.io.read_raw_fif(op.join(fif_path, 'sef_raw_sss.fif'))

Opening raw data file /home/stefanappelhoff/mne_data/MNE-somato-data/MEG/somato/sef_raw_sss.fif...
    Range : 237600 ... 506999 =    791.189 ...  1688.266 secs
Ready.
Current compensation grade : 0


In [4]:
# Set the sex of the participant to male
# Subject sex (0=unknown, 1=male, 2=female).
raw.info['subject_info']['sex'] = raw.info['subject_info'].get('sex', 1)

In [5]:
# Get the events, and assign an ID
events = mne.find_events(raw)
event_id = dict()
for val in np.unique(events[:, -1]):
    event_id['somato_event{}'.format(val)] = val
    
event_id

111 events found
Event IDs: [1]


{'somato_event1': 1}

In [6]:
# Make a bids name for the raw file
bids_basename = make_bids_basename(subject='01', task='somato')

In [7]:
# Make a location for the BIDS formatted data
somato_parent, somato_name = op.split(somato.data_path())
output_path = op.join(somato_parent, 'bids_' + somato_name)

In [8]:
# Write the data
write_raw_bids(raw, bids_basename, output_path, events, event_id, overwrite=True)

Opening raw data file /home/stefanappelhoff/mne_data/MNE-somato-data/MEG/somato/sef_raw_sss.fif...
    Range : 237600 ... 506999 =    791.189 ...  1688.266 secs
Ready.
Current compensation grade : 0
Creating folder: /home/stefanappelhoff/mne_data/bids_MNE-somato-data/sub-01/meg

Writing '/home/stefanappelhoff/mne_data/bids_MNE-somato-data/participants.tsv'...

participant_id	age	sex
sub-01	n/a	M

Writing '/home/stefanappelhoff/mne_data/bids_MNE-somato-data/participants.json'...

{
    "participant_id": {
        "Description": "Unique participant identifier"
    },
    "age": {
        "Description": "Age of the participant at time of testing",
        "Units": "years"
    },
    "sex": {
        "Description": "Biological sex of the participant",
        "Levels": {
            "F": "female",
            "M": "male"
        }
    }
}

Writing '/home/stefanappelhoff/mne_data/bids_MNE-somato-data/sub-01/sub-01_scans.tsv'...

filename	acq_time
meg/sub-01_task-somato_meg.fif	2007-07-05T13

/home/stefanappelhoff/Desktop/bids/mne-bids/mne_bids/write.py:405: UserWarning: No line frequency found, defaulting to 50 Hz
  warn('No line frequency found, defaulting to 50 Hz')


Closing /home/stefanappelhoff/mne_data/bids_MNE-somato-data/sub-01/meg/sub-01_task-somato_meg.fif [done]


'/home/stefanappelhoff/mne_data/bids_MNE-somato-data'

In [9]:
# Edit dataset_description.json to includ link to dataset
dataset_description_json = op.join(output_path, 'dataset_description.json')
with open(dataset_description_json, 'r') as fin:
    jsondict = json.load(fin)
    
# Edit some fields we know of
jsondict['Name'] = 'MNE-somato-data-bids'
jsondict['ReferencesAndLinks'] = ['http://martinos.org/mne/stable/manual/datasets_index.html?#somatosensory']
jsondict['Acknowledgements'] = "Stefan Appelhoff, Alexandre Gramfort, and Mainak Jas formatted these data to BIDS."
    
    
# overwrite the json, updating it
with open(dataset_description_json, 'w') as fout:
    json.dump(jsondict, fout,indent=2, sort_keys=True)

In [10]:
# Write anatomical data to BIDS
# we take the original T1 from the freesurfer directory
t1w = op.join(somato.data_path(), 'subjects', 'somato', 'mri', 'T1.mgz')

# we also take the trans file, and use it to write the coordinates of 
# anatomical landmarks to a T1w.json file
trans = op.join(somato.data_path(), 'MEG', 'somato', 'sef_raw_sss-trans.fif')

# Copy over the MRI, convert it to NIfTI format, and write the anatomical landmarks
# in voxel coordinates
write_anat(output_path, subject='01', t1w=t1w, trans=trans, raw=raw,
           overwrite=True, verbose=True)


Writing '/home/stefanappelhoff/mne_data/bids_MNE-somato-data/sub-01/anat/sub-01_T1w.json'...

{
    "AnatomicalLandmarkCoordinates": {
        "LPA": [
            198.40044760003894,
            155.54004821809775,
            107.97464085171057
        ],
        "NAS": [
            130.7952568859909,
            120.6312019648147,
            211.33242222532203
        ],
        "RPA": [
            52.85523322409679,
            149.5402399941663,
            111.46539702607649
        ]
    }
}


'/home/stefanappelhoff/mne_data/bids_MNE-somato-data/sub-01/anat'

In [11]:
# Show what the converted data looks like so far
print_dir_tree(output_path)

|bids_MNE-somato-data
|--- participants.tsv
|--- dataset_description.json
|--- README
|--- participants.json
|--- derivatives
|------ mne_forward_model
|--------- sub-01
|------------ sub-01_task-somato-meg-oct-6-fwd.fif
|------ freesurfer
|--------- subjects
|--- sub-01
|------ sub-01_scans.tsv
|------ meg
|--------- sub-01_task-somato_channels.tsv
|--------- sub-01_task-somato_meg.json
|--------- sub-01_task-somato_events.tsv
|--------- sub-01_task-somato_meg.fif
|--------- sub-01_coordsystem.json
|------ anat
|--------- sub-01_T1w.nii.gz
|--------- sub-01_T1w.json


# Add derivatives

some files cannot be specified in BIDS yet. We add these to derivatives


In [12]:
# General derivatives directory
derivatives_dir =op.join(output_path, 'derivatives')
if not op.exists(derivatives_dir):
    os.makedirs(derivatives_dir)

##### Notes on freesurfer

we *cannot* simply copy over the freesurfer files from the non-BIDS somato data, because we need to rename the subject to '01' instead of 'somato'

###### Example workflow (bash)

First run freesurfer:

```bash
$ export SUBJECTS_DIR=/home/username/subjects

$ echo $SUBJECTS_DIR
/home/username/subjects

$ recon-all -all -i /home/username/bids_MNE-somato-data/sub-01/anat/sub-01_T1w.nii.gz -s 01
```

Then your `$SUBJECTS_DIR` should contain a folder `01` with the somato MRI data derivatives and you can successfully run the following cell.

In [13]:
# Make a freesurfer directory. 
subjects_dir = op.join(derivatives_dir, 'freesurfer', 'subjects')
if not op.exists(subjects_dir):
    os.makedirs(subjects_dir)
    
    
# The following path MUST point to the subject directory used with the `recon-all`
# freesurfer call on the mri data in "sub-01/anat/sub-01_T1w.nii.gz"
# Example:
# 
# `recon-all -all -i /home/username/bids_MNE-somato-data/sub-01/anat/sub-01_T1w.nii.gz
#  -s 01 -d /home/username/subjects/01`
# `freesurfer_data_to_copy_over = '/home/username/Desktop/subjects/01'`
local_subjects_dir = os.getenv('SUBJECTS_DIR', None)
if local_subjects_dir is not None:
    freesurfer_data_to_copy_overop.join(local_subjects_dir, '01')
    sh.copytree(freesurfer_data_to_copy_over, subjects_dir)
else:
    warnings.warn('\n\n**NOT** copying the freesurfer data. '
                  'Please read the instructions in the cell above.')
    
    

/home/stefanappelhoff/miniconda3/envs/mne/lib/python3.6/site-packages/ipykernel_launcher.py:19: UserWarning: 

**NOT** copying the freesurfer data. Please read the instructions in the cell above.


In [14]:
# Make a directory for the forward model and copy it there
forward_dir = op.join(derivatives_dir, 'mne_forward_model', 'sub-01')
if not op.exists(forward_dir):
    os.makedirs(forward_dir)
    
# copy it, overwriting old if it's there
old_forward = op.join(somato.data_path(), 'MEG', 'somato', 'somato-meg-oct-6-fwd.fif')
bids_forward = op.join(forward_dir, 'sub-01_task-somato-meg-oct-6-fwd.fif')
sh.copyfile(old_forward, bids_forward)

'/home/stefanappelhoff/mne_data/bids_MNE-somato-data/derivatives/mne_forward_model/sub-01/sub-01_task-somato-meg-oct-6-fwd.fif'

In [15]:
# Show what the converted data looks like so far
print_dir_tree(output_path)

|bids_MNE-somato-data
|--- participants.tsv
|--- dataset_description.json
|--- README
|--- participants.json
|--- derivatives
|------ mne_forward_model
|--------- sub-01
|------------ sub-01_task-somato-meg-oct-6-fwd.fif
|------ freesurfer
|--------- subjects
|--- sub-01
|------ sub-01_scans.tsv
|------ meg
|--------- sub-01_task-somato_channels.tsv
|--------- sub-01_task-somato_meg.json
|--------- sub-01_task-somato_events.tsv
|--------- sub-01_task-somato_meg.fif
|--------- sub-01_coordsystem.json
|------ anat
|--------- sub-01_T1w.nii.gz
|--------- sub-01_T1w.json


# Create READMEs and copy over this code as well

In [16]:
# Prepare the text for the central README
txt1 = """MNE-somato-data-bids
====================

This dataset contains the MNE-somato-data in BIDS format.

The conversion can be reproduced through the jupyter notebook stored in the
/code directory of this dataset. See the README in that directory.

The /derivaties directory contains the outputs of running the freesurfer pipeline
`recon-all` on the MRI data with no additional commandline options (only defaults were used).

It also contains, the forward model *-fwd.fif, which was produced using the source space
definition, a -trans.fif file, and the boundary element model / conductor model that live
in freesurfer/subjects/01/bem/*-bem-sol.fif.

The -trans.fif file is not saved, but can be recovered from the anatomical landmarks
in the sub-01/anat/T1w.json file and MNE-BIDS' function `get_head_mri_transform`.

See: https://github.com/mne-tools/mne-bids for more information.


Note on freesurfer
==================
the freesurfer pipeline `recon-all` was run new for the sake of converting the somato
data to BIDS format. This needed to be done to change the "somato" subject name to 
the BIDS subject label "01".


"""

In [17]:
readme_fname = op.join(output_path, 'README')

with open(readme_fname, 'w') as fout:
    print(txt1, file=fout)

In [24]:
# Prepare a /code directory and write a README
code_dir = op.join(output_path, 'code')
if not op.exists(code_dir):
    os.makedirs(code_dir)

txt2 = """`convert_somato_data.ipynb` is a [jupyter notebook](https://jupyter.org/)
that helps to convert the MNE-somato-data into [BIDS](bids.neuroimaging.io) format.

To run this notebook, you can follow the instructions here: https://jupyter.org/install
"""

readme_fname = op.join(code_dir, 'README')

with open(readme_fname, 'w') as fout:
    print(txt2, file=fout)


In [ ]:
# Finally, copy over this jupyter notebook as well

In [29]:
fname = 'convert_somato_data.ipynb'
fpath = os.path.dirname(os.path.realpath('__file__'))
fpath = op.join(fpath, fname)

In [31]:

sh.copyfile(fpath, op.join(code_dir, fname))

IsADirectoryError: [Errno 21] Is a directory: '/home/stefanappelhoff/mne_data/bids_MNE-somato-data/code'

In [18]:
# Show what the converted data looks like so far
print_dir_tree(output_path)

|bids_MNE-somato-data
|--- participants.tsv
|--- dataset_description.json
|--- README
|--- participants.json
|--- derivatives
|------ mne_forward_model
|--------- sub-01
|------------ sub-01_task-somato-meg-oct-6-fwd.fif
|------ freesurfer
|--------- subjects
|--- sub-01
|------ sub-01_scans.tsv
|------ meg
|--------- sub-01_task-somato_channels.tsv
|--------- sub-01_task-somato_meg.json
|--------- sub-01_task-somato_events.tsv
|--------- sub-01_task-somato_meg.fif
|--------- sub-01_coordsystem.json
|------ anat
|--------- sub-01_T1w.nii.gz
|--------- sub-01_T1w.json
